In [1]:
# Set up logging

import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
logging.getLogger("council").setLevel(logging.DEBUG)

In [2]:
# Council imports

from council.runners import Budget
from council.contexts import AgentContext, ChatHistory
from council.agents import Agent
from council.chains import Chain
from council.llm.openai_llm_configuration import OpenAILLMConfiguration
from council.llm.openai_llm import OpenAILLM

from skills import SectionWriterSkill, OutlineWriterSkill
from controller import WritingAssistantController
from evaluator import BasicEvaluatorWithSource

In [3]:
# Read environment variables and initialize OpenAI LLM instance

import dotenv
dotenv.load_dotenv()
openai_llm = OpenAILLM(config=OpenAILLMConfiguration.from_env())

In [4]:
# Create Skills

outline_skill = OutlineWriterSkill(openai_llm)
writing_skill = SectionWriterSkill(openai_llm)

In [5]:
# Create Chains

outline_chain = Chain(
    name="Outline Writer",
    description="Write or revise the outline (i.e. section headers) of a research article in markdown format. Only use this chain when the ARTICLE OUTLINE is missing or incomplete.",
    runners=[outline_skill]
)

writer_chain = Chain(
    name="Article Writer",
    description="Write or revise specific section bodies of a research article in markdown format. Use this chain to write the main research article content.",
    runners=[writing_skill]
)

In [6]:
# Create Controller

controller = WritingAssistantController(
    openai_llm,
    top_k_execution_plan=3
)

In [7]:
# Initialize Agent

chat_history = ChatHistory()
run_context = AgentContext(chat_history)
agent = Agent(controller, [outline_chain, writer_chain], BasicEvaluatorWithSource())

In [8]:
# Run Agent

chat_history.add_user_message("Write a detailed research article about the history of video games.")
result = agent.execute(run_context, Budget(1200))

[2023-07-26 08:46:16-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-26 08:46:16-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="1"
[2023-07-26 08:46:16-0400 DEBUG MainThread council.llm.llm_base:post_chat_request:30] message="starting execution of llm request"
[2023-07-26 08:46:16-0400 DEBUG MainThread council.llm.openai_chat_completions_llm:_post_request:145] message="Sending chat GPT completions request" payload="{'temperature': 0.0, 'n': 1, 'model': 'gpt-4', 'messages': [{'role': 'system', 'content': 'You are the Controller module for an AI assistant built to write and revise research articles.'}, {'role': 'user', 'content': "\n        # Task Description\n        Your task is to decide how best to write or revise the ARTICLE. Considering the ARTICLE OUTLINE, ARTICLE, and the CONVERSATION HISTORY,\n        use your avaiable CHAINS to decide what steps to take next. You are not respons

CONTROLLER
Outline Writer;10;Create an outline for the research article about the history of video games. The outline should include sections such as Introduction, Early History, Evolution of Video Games, Impact on Society, and Conclusion.
Article Writer;8;Once the outline is ready, start writing the Introduction section, providing a brief overview of the topic and what the article will cover.


[2023-07-26 08:46:38-0400 DEBUG chain_Outline Writer_0 council.llm.openai_chat_completions_llm:_post_request:151] message="got chat GPT completions result" id="chatcmpl-7gYCGXGqN9WbFYQ39CFS8EQguOgeu" model="gpt-4-0613" prompt_tokens="242" total_tokens="414" completion_tokens="172"
[2023-07-26 08:46:38-0400 DEBUG chain_Outline Writer_0 council.llm.llm_base:post_chat_request:37] message="done execution of llm request"
[2023-07-26 08:46:38-0400 INFO chain_Outline Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="OutlineWriterSkill" skill_message="I've edited the outline and placed the response in the 'data' field."
[2023-07-26 08:46:38-0400 DEBUG MainThread council.runners.runner_base:run:51] done running OutlineWriterSkill
[2023-07-26 08:46:38-0400 INFO MainThread council.agents.agent:execute:76] message="chain execution ended" chain="Outline Writer" execution_unit="Outline Writer: Create an outline for the research article about the history of v

OUTLINE
# Introduction
- Brief overview of the topic
- Importance of understanding the history of video games

# Early History of Video Games
- The first video games: origins and pioneers
- The birth of arcade games
- The advent of home consoles

# Evolution of Video Games
- Transition from 2D to 3D gaming
- The rise of online gaming
- The impact of mobile gaming
- The advent of virtual and augmented reality games

# Impact on Society
- The economic impact of the video game industry
- The influence of video games on popular culture
- The role of video games in education and skill development
- Controversies and criticisms: violence, addiction, and social isolation

# Conclusion
- Recap of the evolution and impact of video games
- The future of video games: trends and predictions
```
ARTICLE
# The History of Video Games: A Detailed Exploration

## Introduction

The history of video games is a fascinating journey that spans over half a century, from the rudimentary designs of the 1950s t

[2023-07-26 08:48:17-0400 DEBUG MainThread council.llm.openai_chat_completions_llm:_post_request:151] message="got chat GPT completions result" id="chatcmpl-7gYDkefNGozpGGj8m9aALxbnUznMD" model="gpt-4-0613" prompt_tokens="990" total_tokens="1243" completion_tokens="253"
[2023-07-26 08:48:17-0400 DEBUG MainThread council.llm.llm_base:post_chat_request:37] message="done execution of llm request"
[2023-07-26 08:48:17-0400 DEBUG MainThread council:get_plan:108] controller get_plan response: Article Writer;8;Revise the section "The Early Beginnings" to match the outline's title "Early History of Video Games" and include details about the origins and pioneers of video games, the birth of arcade games, and the advent of home consoles.
Article Writer;9;Revise the section "The Rise of Home Consoles and 3D Gaming" to match the outline's title "Evolution of Video Games" and include details about the transition from 2D to 3D gaming, the rise of online gaming, the impact of mobile gaming, and the a

...

OUTLINE
# Introduction
- Brief overview of the topic
- Importance of understanding the history of video games

# Early History of Video Games
- The first video games: origins and pioneers
- The birth of arcade games
- The advent of home consoles

# Evolution of Video Games
- Transition from 2D to 3D gaming
- The rise of online gaming
- The impact of mobile gaming
- The advent of virtual and augmented reality games

# Impact on Society
- The economic impact of the video game industry
- The influence of video games on popular culture
- The role of video games in education and skill development
- Controversies and criticisms: violence, addiction, and social isolation

# Conclusion
- Recap of the evolution and impact of video games
- The future of video games: trends and predictions
```
ARTICLE
# The History of Video Games: A Detailed Exploration

## Introduction

The history of video games is a fascinating journey that spans over half a century, from the rudimentary designs of the 1950s t

[2023-07-26 08:51:51-0400 DEBUG MainThread council.llm.openai_chat_completions_llm:_post_request:151] message="got chat GPT completions result" id="chatcmpl-7gYHGoTVjX4WslSsLdn4sQRCa4qCy" model="gpt-4-0613" prompt_tokens="1416" total_tokens="1634" completion_tokens="218"
[2023-07-26 08:51:51-0400 DEBUG MainThread council.llm.llm_base:post_chat_request:37] message="done execution of llm request"
[2023-07-26 08:51:51-0400 DEBUG MainThread council:get_plan:108] controller get_plan response: Article Writer;8;Revise the section "The Early Beginnings" to match the outline header "Early History of Video Games" and include subtopics such as "The first video games: origins and pioneers", "The birth of arcade games", and "The advent of home consoles".
Article Writer;9;Revise the section "The Rise of Home Consoles and 3D Gaming" to match the outline header "Evolution of Video Games" and include subtopics such as "Transition from 2D to 3D gaming", "The rise of online gaming", "The impact of mobile

CONTROLLER
Article Writer;9;Revise the section "The Rise of Home Consoles and 3D Gaming" to match the outline header "Evolution of Video Games" and include subtopics such as "Transition from 2D to 3D gaming", "The rise of online gaming", "The impact of mobile gaming", and "The advent of virtual and augmented reality games".
Article Writer;8;Revise the section "The Early Beginnings" to match the outline header "Early History of Video Games" and include subtopics such as "The first video games: origins and pioneers", "The birth of arcade games", and "The advent of home consoles".
Article Writer;7;Revise the section "Societal Implications of Video Games" to match the outline header "Impact on Society" and include subtopics such as "The economic impact of the video game industry", "The influence of video games on popular culture", "The role of video games in education and skill development", and "Controversies and criticisms: violence, addiction, and social isolation".


[2023-07-26 08:52:51-0400 DEBUG chain_Article Writer_0 council.llm.openai_chat_completions_llm:_post_request:151] message="got chat GPT completions result" id="chatcmpl-7gYHXrIYWBU25vdJ6tEUZyFIA9R4t" model="gpt-4-0613" prompt_tokens="1372" total_tokens="2197" completion_tokens="825"
[2023-07-26 08:52:51-0400 DEBUG chain_Article Writer_0 council.llm.llm_base:post_chat_request:37] message="done execution of llm request"
[2023-07-26 08:52:51-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="SectionWriterSkill" skill_message="I've written or edited the article and placed it in the 'data' field."
[2023-07-26 08:52:51-0400 DEBUG MainThread council.runners.runner_base:run:51] done running SectionWriterSkill
[2023-07-26 08:52:51-0400 INFO MainThread council.agents.agent:execute:76] message="chain execution ended" chain="Article Writer" execution_unit="Article Writer: Revise the section "The Rise of Home Consoles and 3D Gaming" t

OUTLINE
# Introduction
- Brief overview of the topic
- Importance of understanding the history of video games

# Early History of Video Games
- The first video games: origins and pioneers
- The birth of arcade games
- The advent of home consoles

# Evolution of Video Games
- Transition from 2D to 3D gaming
- The rise of online gaming
- The impact of mobile gaming
- The advent of virtual and augmented reality games

# Impact on Society
- The economic impact of the video game industry
- The influence of video games on popular culture
- The role of video games in education and skill development
- Controversies and criticisms: violence, addiction, and social isolation

# Conclusion
- Recap of the evolution and impact of video games
- The future of video games: trends and predictions
```
ARTICLE
# The History of Video Games: A Detailed Exploration

## Introduction

The history of video games is a fascinating journey that spans over half a century, from the rudimentary designs of the 1950s t

In [9]:
print(result.messages[-1].message.message)

# The History of Video Games: A Detailed Exploration

## Introduction

The history of video games is a fascinating journey that spans over half a century, from the rudimentary designs of the 1950s to the sophisticated, immersive games of the 21st century. This article will delve into the evolution of video games, tracing their origins, the technological advancements that have shaped them, and the cultural impact they have had over the years.

## The Early Beginnings

We will explore the early beginnings of video games, starting with the creation of the first known video game, "Tennis for Two," in 1958. Developed by physicist William Higinbotham, this game, though simple by today's standards, marked the beginning of a new era in entertainment.

## The Birth of the Arcade Era

The 1970s marked the birth of the arcade era, with the release of iconic games like "Pong" and "Space Invaders." These games, developed by pioneers such as Nolan Bushnell and Toshihiro Nishikado, not only revolutio

In [10]:
chat_history.add_user_message("Can you please add more details in each section?")
result = agent.execute(run_context, Budget(600))

[2023-07-26 08:56:46-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-26 08:56:46-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="4"
[2023-07-26 08:56:46-0400 DEBUG MainThread council.llm.llm_base:post_chat_request:30] message="starting execution of llm request"
[2023-07-26 08:56:46-0400 DEBUG MainThread council.llm.openai_chat_completions_llm:_post_request:145] message="Sending chat GPT completions request" payload="{'temperature': 0.0, 'n': 1, 'model': 'gpt-4', 'messages': [{'role': 'system', 'content': 'You are the Controller module for an AI assistant built to write and revise research articles.'}, {'role': 'user', 'content': '\n        # Task Description\n        Your task is to decide how best to write or revise the ARTICLE. Considering the ARTICLE OUTLINE, ARTICLE, and the CONVERSATION HISTORY,\n        use your avaiable CHAINS to decide what steps to take next. You are not respons

CONTROLLER
Article Writer;10;Expand the "Evolution of Video Games" section by adding more details about the transition from 2D to 3D gaming, the rise of online gaming, the impact of mobile gaming, and the advent of virtual and augmented reality games.
Article Writer;9;In the "Early History of Video Games" section, provide more information about the first video games, the birth of arcade games, and the advent of home consoles.
Article Writer;9;In the "Impact on Society" section, elaborate more on the economic impact of the video game industry, the influence of video games on popular culture, the role of video games in education and skill development, and controversies and criticisms related to violence, addiction, and social isolation.
Article Writer;8;Please add more details to the "Introduction" section, specifically about the importance of understanding the history of video games.
Article Writer;8;In the "Conclusion" section, provide a more detailed recap of the evolution and impact 

[2023-07-26 08:58:48-0400 DEBUG chain_Article Writer_0 council.llm.openai_chat_completions_llm:_post_request:151] message="got chat GPT completions result" id="chatcmpl-7gYMc7dJVjmyxeV5W1og2F4Tv4tzd" model="gpt-4-0613" prompt_tokens="1525" total_tokens="2817" completion_tokens="1292"
[2023-07-26 08:58:48-0400 DEBUG chain_Article Writer_0 council.llm.llm_base:post_chat_request:37] message="done execution of llm request"
[2023-07-26 08:58:48-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="SectionWriterSkill" skill_message="I've written or edited the article and placed it in the 'data' field."
[2023-07-26 08:58:48-0400 DEBUG MainThread council.runners.runner_base:run:51] done running SectionWriterSkill
[2023-07-26 08:58:48-0400 INFO MainThread council.agents.agent:execute:76] message="chain execution ended" chain="Article Writer" execution_unit="Article Writer: Expand the "Evolution of Video Games" section by adding more 

OUTLINE
# Introduction
- Brief overview of the topic
- Importance of understanding the history of video games

# Early History of Video Games
- The first video games: origins and pioneers
- The birth of arcade games
- The advent of home consoles

# Evolution of Video Games
- Transition from 2D to 3D gaming
- The rise of online gaming
- The impact of mobile gaming
- The advent of virtual and augmented reality games

# Impact on Society
- The economic impact of the video game industry
- The influence of video games on popular culture
- The role of video games in education and skill development
- Controversies and criticisms: violence, addiction, and social isolation

# Conclusion
- Recap of the evolution and impact of video games
- The future of video games: trends and predictions
```
ARTICLE
# The History of Video Games: A Detailed Exploration

## Introduction

The history of video games is a fascinating journey that spans over half a century, from the rudimentary designs of the 1950s t

In [11]:
print(result.messages[-1].message.message)

# The History of Video Games: A Detailed Exploration

## Introduction

The history of video games is a fascinating journey that spans over half a century, from the rudimentary designs of the 1950s to the sophisticated, immersive games of the 21st century. This article will delve into the evolution of video games, tracing their origins, the technological advancements that have shaped them, and the cultural impact they have had over the years.

## Early History of Video Games

### The First Video Games: Origins and Pioneers

The first known video game, "Tennis for Two," was created in 1958 by physicist William Higinbotham. This game, though simple by today's standards, marked the beginning of a new era in entertainment. It was designed to be played on an oscilloscope, a piece of electronic test equipment. The game simulated a game of tennis or ping-pong on a two-dimensional screen. Following this, in 1962, a game called "Spacewar!" was developed by Steve Russell. It was the first game in